In [1]:
import numpy as np
import tensorflow as tf
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt

from IPython.display import Image
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils

from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import Dense, Activation
import pickle

%matplotlib inline

#### Loading dataset into train, valid, and test dirs

In [5]:
os.chdir("Sign-Language-Digits-Dataset-master/Dataset")
if os.path.isdir('train/0/') is False:
    os.mkdir('train')
    os.mkdir('valid')
    os.mkdir('test')
    
    for i in range(0,10):
        shutil.move(f'{i}', 'train')
        os.mkdir(f'valid/{i}')
        os.mkdir(f'test/{i}')
                 
        valid_samples = random.sample(os.listdir(f'train/{i}'), 30)
        for j in valid_samples:
            shutil.move(f'train/{i}/{j}', f'valid/{i}')
                 
        test_samples = random.sample(os.listdir(f'train/{i}'), 5)
        for k in test_samples:
            shutil.move(f'train/{i}/{k}', f'test/{i}')
        
os.chdir('../..')

In [6]:
for i in range(0, 10):
    assert len(os.listdir(f'Sign-Language-Digits-Dataset-master/Dataset/valid/{i}')) == 30
    assert len(os.listdir(f'Sign-Language-Digits-Dataset-master/Dataset/test/{i}')) == 5

In [7]:
train_path = 'Sign-Language-Digits-Dataset-master/Dataset/train/'
valid_path = 'Sign-Language-Digits-Dataset-master/Dataset/valid/'
test_path = 'Sign-Language-Digits-Dataset-master/Dataset/test/'

In [8]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)\
    .flow_from_directory(directory=train_path, target_size=(224,224), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
    .flow_from_directory(directory=valid_path, target_size=(224,224), batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
    .flow_from_directory(directory=test_path, target_size=(224,224), batch_size=10, shuffle=False)

Found 1712 images belonging to 10 classes.
Found 300 images belonging to 10 classes.
Found 50 images belonging to 10 classes.


In [14]:
assert train_batches.n == 1712
assert valid_batches.n == 300
assert test_batches.n == 50
# assert train_batches == valid_batches == test_batches == 10

In [15]:
mobile = tf.keras.applications.mobilenet.MobileNet()

In [16]:
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [17]:
params = count_params(mobile)
assert params['non_trainable_params'] == 21888
assert params['trainable_params'] == 4231976

In [18]:
x = mobile.layers[-6].output
output = Dense(units=10, activation='softmax')(x)

In [19]:
model = Model(inputs=mobile.input, outputs=output)

In [20]:
for layer in model.layers[:-20]:
    layer.trainable = False

In [21]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)     

In [22]:
params = count_params(mobile)
assert params['non_trainable_params'] == 1365184
assert params['trainable_params'] == 1873930

#### training the model

In [23]:
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [2]:
model.fit(x=train_batches, validation=valid_batches, epochs=10, verbose=2)

In [ ]:
model.pickle()

#### Predicting the sign language digits

In [28]:
test_labels = test_batches.classes

In [29]:
predictions = model.predict(test_batches, verbose =0)

In [3]:
cm = confusion_matrix(y_true = test_labels, y_predict = predictions.argmax(axis=1))

In [ ]:
plot_confusion_matrix(cm, ccm_plot_labels, title="confusion_matrix")